In [1]:
#%pip install git+https://github.com/ultralytics/ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics 8.3.50  Python-3.9.13 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
Setup complete  (16 CPUs, 31.9 GB RAM, 208.0/265.6 GB disk)


# Test Results

In [1]:
# Path to the test images
test_images_dir = 'Dataset/test/images'

In [2]:
# Path to data.yaml
data_yaml_path = 'data.yaml'

In [4]:
import time
import torch
import os
from ultralytics import YOLO

# Define the models and their paths
models_info = [
    {
        "name": "Model_N_best.pt.pt",
        "weights_path": "Model/Model_N_best.pt"
    },
    {
        "name": "Model_S_best.pt.pt",
        "weights_path": "Model/Model_S_best.pt"
    },
    {
        "name": "Model_M_best.pt",
        "weights_path": "Model/Model_M_best.pt"
    }
]


# # Path to data.yaml
# data_yaml_path = 'data.yaml'


# Get list of image files in the test directory
img_files = [f for f in os.listdir(test_images_dir) if f.endswith(('png', 'jpg', 'jpeg'))]
num_images = len(img_files)

# Initialize a list to store results
results = []

# Iterate over each model and evaluate
for model_info in models_info:
    model_name = model_info["name"]
    weights_path = model_info["weights_path"]
    
    # Load the YOLO model
    model = YOLO(weights_path)
    
    # -------------------- Model Evaluation --------------------
    # Evaluate the model on the test set
    metrics = model.val(data=data_yaml_path, split="test", batch=16, imgsz=640)

    # Extract metrics safely
    try:
        precision = metrics.box.p.mean()  # Mean Precision
        recall = metrics.box.r.mean()     # Mean Recall
        map50 = metrics.box.map50.mean()  # Mean mAP@50
        map = metrics.box.map.mean()      # Mean mAP@50-95
    except AttributeError:
        print(f"Error extracting metrics for {model_name}. Skipping...")
        continue

    # -------------------- Inference Time Calculation --------------------
    total_time = 0

    # Loop through each image file and perform inference
    for img_file in img_files:
        img_path = os.path.join(test_images_dir, img_file)

        # Suppress output from YOLO during inference
        with torch.no_grad():
            start_time = time.time()
            _ = model(img_path, verbose=False)  # Perform inference with verbose=False
            inference_time = time.time() - start_time

        # Accumulate the inference time in milliseconds
        total_time += inference_time * 1000  # Convert to milliseconds

    # Calculate the average inference time in milliseconds
    average_inference_time = total_time / num_images

    # -------------------- Model Parameters and FLOPs --------------------
    # Extracting model parameters (in millions)
    params = sum(p.numel() for p in model.parameters()) / 1e6  # In millions

    # Assuming the model's FLOPs is roughly related to the number of parameters (this is an approximation)
    # Using a simple heuristic: FLOPs (billions) ~ Parameters (millions) * 2 (simplified estimate)
    flops = params * 2  # This is a rough approximation and may differ based on the architecture

    # Append the results for this model
    results.append({
        "Model": model_name,
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "mAP@50": round(map50, 3),
        "mAP@50-95": round(map, 3),
        "Params(M)": round(params, 3),
        "FLOPs(B)": round(flops, 3),
        "Inference Time (ms)": round(average_inference_time, 2)
    })

# Print the results table
print(f"\n{'Model':<30}{'Precision':<10}{'Recall':<10}{'mAP@50':<10}{'mAP@50-95':<15}{'Params(M)':<10}{'FLOPs(B)':<10}{'Inference Time (ms)'}")
print("-" * 100)
for result in results:
    print(f"{result['Model']:<30}{result['Precision']:<10}{result['Recall']:<10}{result['mAP@50']:<10}{result['mAP@50-95']:<15}{result['Params(M)']:<10}{result['FLOPs(B)']:<10}{result['Inference Time (ms)']}")

Ultralytics 8.3.50  Python-3.9.13 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
YOLO11n summary (fused): 238 layers, 2,583,907 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:12<00:00,  9.90it/s]


                   all       2033       4720      0.717      0.532      0.544      0.346
           boneanomaly         16         19      0.289      0.211      0.178     0.0868
            bonelesion          8          8      0.769      0.125      0.171     0.0864
           foreignbody          1          1          1          0          0          0
              fracture       1351       1784      0.856      0.886      0.932      0.567
                 metal         74         85       0.83      0.941      0.952      0.805
    periostealreaction        230        352      0.584      0.639      0.637      0.291
          pronatorsign         47         47       0.72      0.438      0.587      0.325
            softtissue         38         39      0.447      0.564      0.446      0.211
                  text       2029       2385      0.962      0.982      0.992      0.742
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect

val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:11<00:00, 10.73it/s]


                   all       2033       4720       0.78      0.556      0.692       0.44
           boneanomaly         16         19      0.435      0.158       0.17     0.0902
            bonelesion          8          8      0.868       0.25      0.305      0.205
           foreignbody          1          1          1          0      0.995      0.502
              fracture       1351       1784      0.892      0.918      0.956      0.606
                 metal         74         85      0.933      0.965      0.974      0.846
    periostealreaction        230        352      0.722      0.641      0.736      0.371
          pronatorsign         47         47      0.566      0.702      0.709      0.411
            softtissue         38         39      0.632      0.385      0.386      0.179
                  text       2029       2385      0.971      0.985      0.993      0.748
Speed: 0.1ms preprocess, 1.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect

val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:15<00:00,  8.04it/s]


                   all       2033       4720      0.843       0.68      0.836       0.57
           boneanomaly         16         19       0.81      0.526      0.615      0.431
            bonelesion          8          8      0.926      0.625      0.735      0.463
           foreignbody          1          1          1          0      0.995      0.597
              fracture       1351       1784      0.931       0.95      0.977      0.666
                 metal         74         85       0.96          1      0.995      0.884
    periostealreaction        230        352      0.752      0.699      0.775      0.439
          pronatorsign         47         47      0.699      0.745      0.818      0.508
            softtissue         38         39      0.533       0.59      0.619      0.392
                  text       2029       2385      0.976      0.987      0.994      0.755
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect

In [5]:
import time
import torch
import os
from ultralytics import YOLO

# Define the models and their paths
models_info = [
    {
        "name": "Model_m_640_32_Adam_best.pt",
        "weights_path": "Model/Model_m_640_32_Adam_best.pt"
    },
    {
        "name": "Model_m_640_16_Adam_best.pt",
        "weights_path": "Model/Model_m_640_16_Adam_best.pt"
    },
    {
        "name": "Model_m_640_SGD_32_best.pt",
        "weights_path": "Model/Model_m_640_32_SGD_best.pt"
    },
    {
        "name": "Model_m_640_16_SGD_best.pt",
        "weights_path": "Model/Model_m_640_16_SGD_best.pt"
    }
]

# # Path to data.yaml
# data_yaml_path = 'data.yaml'

# # Path to the test images
# test_images_dir = 'Dataset/test/images'

# Get list of image files in the test directory
img_files = [f for f in os.listdir(test_images_dir) if f.endswith(('png', 'jpg', 'jpeg'))]
num_images = len(img_files)

# Initialize a list to store results
results = []

# Iterate over each model and evaluate
for model_info in models_info:
    model_name = model_info["name"]
    weights_path = model_info["weights_path"]
    
    # Load the YOLO model
    model = YOLO(weights_path)
    
    # -------------------- Model Evaluation --------------------
    # Evaluate the model on the test set
    metrics = model.val(data=data_yaml_path, split="test", imgsz=640)

    # Extract metrics safely
    try:
        precision = metrics.box.p.mean()  # Mean Precision
        recall = metrics.box.r.mean()     # Mean Recall
        map50 = metrics.box.map50.mean()  # Mean mAP@50
        map = metrics.box.map.mean()      # Mean mAP@50-95
    except AttributeError:
        print(f"Error extracting metrics for {model_name}. Skipping...")
        continue

    # -------------------- Inference Time Calculation --------------------
    total_time = 0

    # Loop through each image file and perform inference
    for img_file in img_files:
        img_path = os.path.join(test_images_dir, img_file)

        # Suppress output from YOLO during inference
        with torch.no_grad():
            start_time = time.time()
            _ = model(img_path, verbose=False)  # Perform inference with verbose=False
            inference_time = time.time() - start_time

        # Accumulate the inference time in milliseconds
        total_time += inference_time * 1000  # Convert to milliseconds

    # Calculate the average inference time in milliseconds
    average_inference_time = total_time / num_images

    # -------------------- Model Parameters and FLOPs --------------------
    # Extracting model parameters (in millions)
    params = sum(p.numel() for p in model.parameters()) / 1e6  # In millions

    # Assuming the model's FLOPs is roughly related to the number of parameters (this is an approximation)
    # Using a simple heuristic: FLOPs (billions) ~ Parameters (millions) * 2 (simplified estimate)
    flops = params * 2  # This is a rough approximation and may differ based on the architecture

    # Append the results for this model
    results.append({
        "Model": model_name,
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "mAP@50": round(map50, 3),
        "mAP@50-95": round(map, 3),
        "Params(M)": round(params, 3),
        "FLOPs(B)": round(flops, 3),
        "Inference Time (ms)": round(average_inference_time, 2)
    })

# Print the results table
print(f"\n{'Model':<30}{'Precision':<10}{'Recall':<10}{'mAP@50':<10}{'mAP@50-95':<15}{'Params(M)':<10}{'FLOPs(B)':<10}{'Inference Time (ms)'}")
print("-" * 100)
for result in results:
    print(f"{result['Model']:<30}{result['Precision']:<10}{result['Recall']:<10}{result['mAP@50']:<10}{result['mAP@50-95']:<15}{result['Params(M)']:<10}{result['FLOPs(B)']:<10}{result['Inference Time (ms)']}")


Ultralytics 8.3.50  Python-3.9.13 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
YOLO11m summary (fused): 303 layers, 20,036,971 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:15<00:00,  8.22it/s]


                   all       2033       4720      0.441      0.421      0.418      0.233
           boneanomaly         16         19          0          0     0.0145    0.00563
            bonelesion          8          8          1          0      0.126     0.0881
           foreignbody          1          1          0          0          0          0
              fracture       1351       1784       0.76      0.888      0.912      0.541
                 metal         74         85      0.415      0.812      0.691      0.291
    periostealreaction        230        352      0.346      0.716      0.521      0.208
          pronatorsign         47         47      0.416      0.362      0.421       0.19
            softtissue         38         39     0.0859     0.0256     0.0834     0.0369
                  text       2029       2385      0.951      0.983      0.991      0.737
Speed: 0.1ms preprocess, 3.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect

val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:15<00:00,  8.42it/s]


                   all       2033       4720      0.759      0.398      0.447      0.243
           boneanomaly         16         19          1          0     0.0353      0.012
            bonelesion          8          8          1          0      0.126     0.0634
           foreignbody          1          1          1          0     0.0553     0.0276
              fracture       1351       1784      0.881      0.856       0.92      0.549
                 metal         74         85      0.492      0.776      0.681      0.269
    periostealreaction        230        352       0.57       0.56      0.564       0.23
          pronatorsign         47         47      0.502      0.258      0.445      0.208
            softtissue         38         39      0.425      0.152      0.209     0.0946
                  text       2029       2385      0.964      0.983      0.992      0.738
Speed: 0.1ms preprocess, 3.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect

val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:15<00:00,  8.39it/s]


                   all       2033       4720      0.794      0.661      0.792      0.547
           boneanomaly         16         19      0.659      0.474        0.5      0.276
            bonelesion          8          8          1      0.447      0.707      0.468
           foreignbody          1          1          1          0      0.995      0.796
              fracture       1351       1784      0.929      0.948      0.977      0.655
                 metal         74         85      0.932      0.953      0.984      0.847
    periostealreaction        230        352      0.779      0.679      0.782      0.448
          pronatorsign         47         47      0.478      0.872      0.782      0.475
            softtissue         38         39      0.396       0.59      0.403      0.204
                  text       2029       2385      0.975      0.987      0.993      0.756
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect

val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:15<00:00,  8.50it/s]


                   all       2033       4720      0.843       0.68      0.836       0.57
           boneanomaly         16         19       0.81      0.526      0.615      0.431
            bonelesion          8          8      0.926      0.625      0.735      0.463
           foreignbody          1          1          1          0      0.995      0.597
              fracture       1351       1784      0.931       0.95      0.977      0.666
                 metal         74         85       0.96          1      0.995      0.884
    periostealreaction        230        352      0.752      0.699      0.775      0.439
          pronatorsign         47         47      0.699      0.745      0.818      0.508
            softtissue         38         39      0.533       0.59      0.619      0.392
                  text       2029       2385      0.976      0.987      0.994      0.755
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect

In [7]:
import torch
from ultralytics import YOLO

# Paths to required files
weights_path = 'Model/Model_m_640_16_SGD_best.pt'
# data_yaml_path = 'data.yaml'

# Load the YOLO model
model = YOLO(weights_path)

# Evaluate the model on the test set
metrics = model.val(data=data_yaml_path, split="test", batch=16, imgsz=640)

# Extract and print metrics
precision = metrics.box.p.mean()  # Mean Precision
recall = metrics.box.r.mean()     # Mean Recall
map50 = metrics.box.map50.mean()  # Mean mAP@50
map = metrics.box.map.mean()      # Mean mAP@50-95

print(f"Precision: {round(precision, 3)}")
print(f"Recall: {round(recall, 3)}")
print(f"mAP@50: {round(map50, 3)}")
print(f"mAP@50-95: {round(map, 3)}")

Ultralytics 8.3.50  Python-3.9.13 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
YOLO11m summary (fused): 303 layers, 20,036,971 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:14<00:00,  8.67it/s]


                   all       2033       4720      0.843       0.68      0.836       0.57
           boneanomaly         16         19       0.81      0.526      0.615      0.431
            bonelesion          8          8      0.926      0.625      0.735      0.463
           foreignbody          1          1          1          0      0.995      0.597
              fracture       1351       1784      0.931       0.95      0.977      0.666
                 metal         74         85       0.96          1      0.995      0.884
    periostealreaction        230        352      0.752      0.699      0.775      0.439
          pronatorsign         47         47      0.699      0.745      0.818      0.508
            softtissue         38         39      0.533       0.59      0.619      0.392
                  text       2029       2385      0.976      0.987      0.994      0.755
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect

# Validation Results

In [3]:
import torch
from ultralytics import YOLO

# Paths to required files
weights_path = 'Model/Model_m_640_16_SGD_best.pt'
# data_yaml_path = 'data.yaml'

# Load the YOLO model
model = YOLO(weights_path)

# Evaluate the model on the test set
metrics = model.val(data=data_yaml_path, split="val", batch=16, imgsz=640)

# Extract and print metrics
precision = metrics.box.p.mean()  # Mean Precision
recall = metrics.box.r.mean()     # Mean Recall
map50 = metrics.box.map50.mean()  # Mean mAP@50
map = metrics.box.map.mean()      # Mean mAP@50-95

print(f"Precision: {round(precision, 3)}")
print(f"Recall: {round(recall, 3)}")
print(f"mAP@50: {round(map50, 3)}")
print(f"mAP@50-95: {round(map, 3)}")

Ultralytics 8.3.50  Python-3.9.13 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
YOLO11m summary (fused): 303 layers, 20,036,971 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning G:\Shahadath\Dataset\val\labels.cache... 4066 images, 4 backgrounds, 0 corrupt: 100%|██████████| 4066/4066 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 255/255 [00:56<00:00,  4.55it/s]


                   all       4066       9500      0.846       0.74      0.829      0.585
           boneanomaly         46         71      0.712      0.522      0.615      0.379
            bonelesion          9          9      0.859      0.556      0.764      0.472
           foreignbody          4          4      0.877          1      0.995      0.837
              fracture       2736       3653      0.954       0.93       0.98      0.664
                 metal        128        148       0.96      0.966      0.991       0.88
    periostealreaction        421        660      0.842      0.617      0.776      0.434
          pronatorsign        125        126      0.818      0.611      0.791      0.495
            softtissue         72         77      0.615      0.481      0.558      0.344
                  text       4055       4752       0.98      0.975      0.993      0.756
Speed: 0.1ms preprocess, 7.0ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect